In [1]:
import os
import time
import datetime

from feat import Detector
from feat.utils.io import get_test_data_path
from feat.plotting import imshow

import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

import pygetwindow as gw
from pynput import keyboard
#import pyautogui
#import keyboard

import cv2
import mss



In [2]:
def show_data_window(Werte):
    plt.close()
    Index = ['anger','disgust', 'fear', 'happiness', 'sadness', 'surprise', 'neutral']
    fig, ax = plt.subplots(num = 'Emotion Analysis')
    ax.bar(Index, Werte)
    ax.set_xlabel('Emotion')
    ax.set_ylabel('Wert')
    ax.set_title('Emotionswerte')

    plt.show()
    

In [3]:
def bring_zoom_meeting_to_front():
    
    # Den Fenstertitel des Zoom-Meetings angeben
    zoom_window_title = "Zoom Meeting"
    
    # Das Zoom-Meeting-Fenster suchen und in den Vordergrund bringen
    zoom_window = gw.getWindowsWithTitle(zoom_window_title)[0]
    #zoom_window.restore()  # Das Fenster wiederherstellen, falls es minimiert ist
    #zoom_window.maximize()  # Das Fenster maximieren
    
    # Die Position und Größe des Zoom-Meeting-Fensters abrufen
    left, top, right, bottom = zoom_window.left, zoom_window.top, zoom_window.right, zoom_window.bottom
    
    #return left, top, right, bottom, 
    return zoom_window


In [4]:
def take_screenshot(zoom_window, anz_screenshots):
    
    # Aktuellen Zeitstempel für den Dateinamen generieren
    timestamp = time.strftime("%Y%m%d-%H%M%S", time.localtime())
    screenshot_list = []
    screenshot_name_list = []
    count = True
    
    monitor = {"top": zoom_window.top, "left": zoom_window.left, "width": (zoom_window.right-zoom_window.left), "height": (zoom_window.bottom - zoom_window.top)}
   
    with mss.mss() as sct:
        while True:
            
            # Screenshot des ausgewählten Bereichs aufnehmen
            #screenshot = pyautogui.screenshot(region=region)
            img = np.array(sct.grab(monitor))
            screenshot_list.append(img)

            screenshot_name = f"screenshot_{timestamp}"
            screenshot_name_list.append(f"screenshot_{timestamp}")

            # Speichern des Screenshots
            # Dateipfad und Name für den Screenshot
            #screenshot_path = f"./data/screenshot_{timestamp}.png"
            #screenshot.save(f"./data/{screenshot_name}.png")
            #print (count)
                   
            if count == anz_screenshots:
                break
            count += 1
        
        
    #print(f"Screenshot saved: screenshot_{timestamp}.png")
    return screenshot_list


In [5]:
def pyFeat(screenshot_list,anz_screenshots):
    detector = Detector( 
	face_model = "retinaface",
	landmark_model = "mobilefacenet",
	au_model = 'xgb',
	emotion_model = "resmasknet",
    Facepose_model = "img2pose",)
    
    vorhersage_list = []
    emotion_sum =[]
    
    for i in screenshot_list:
        frame=i
        detected_faces = detector.detect_faces(frame)
        detected_landmarks = detector.detect_landmarks(frame, detected_faces)
        vorhersage = detector.detect_emotions(frame, detected_faces, detected_landmarks)
        vorhersage_list.append(vorhersage)

    #print(vorhersage_list)

    for au in vorhersage_list:
        for inn in au:
            emotion_sum.append(np.sum(inn, axis = 0))
            #print(inn)
            
    emotion = np.mean(emotion_sum, axis = 0)
    
    #Skalierung der Werte auf 1
    emotion_sk = np.divide(emotion, np.sum(emotion))
    
    print("---------------------------------------------------------------------")
    #print(emotion_sum)
    #print(emotion_sk)
    print("---------------------------------------------------------------------")
    
    return emotion_sk


In [6]:
def visualisieren(Werte):
    
    print(f'die Summe ist {np.sum(Werte)}')
    show_data_window(Werte)
    #print(Werte)
    
    #create_gui(mean)
   

In [7]:
def on_release(key):
    
    global running
    
    if key == keyboard.Key.esc:
        running = False;
        plt.close() 
        
    elif key == keyboard.Key.tab:
        #print("Next emotion analysis")
        plt.close()
        #exit()


In [8]:
def main():        
    
    #Anzahl an Screenshots die Aufgenommen werden sollen
    anz_screenshots = 10
    timestamp = time.strftime("%Y%m%d-%H%M%S", time.localtime())
    while running:

        # Screenshot aufnehmen und speichern
        screenshot_list = []
        region = bring_zoom_meeting_to_front()
        #print(region)
        screenshot_list = take_screenshot(region, anz_screenshots)
        #print (screenshot_list)
        emotion = pyFeat(screenshot_list,anz_screenshots)
        visualisieren(emotion)
        timestamp = time.strftime("%Y%m%d-%H%M%S", time.localtime())
        print(timestamp)
            
    listener.stop()
    print("stopped!")
    

In [ ]:
# Starte das Hauptprogramm

running = True
listener = keyboard.Listener(on_release=on_release)
 # run listener in background so that the while loop gets executed
listener.start()
main()


C:\Users\ferdi\anaconda3\envs\IoT\lib\site-packages\torchvision\transforms\functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


---------------------------------------------------------------------
---------------------------------------------------------------------
die Summe ist 1.0
20230709-094751


C:\Users\ferdi\anaconda3\envs\IoT\lib\site-packages\torchvision\transforms\functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(
